In [1]:
import json,re
plos_datas = []
# with open('./train_plos.jsonl', 'r', encoding="utf-8") as f:
with open('./test_plos.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        plos_datas.append(data)

In [2]:
import json,re
elife_datas = []
# with open('./train_elife.jsonl', 'r', encoding="utf-8") as f:
with open('./test_elife.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        elife_datas.append(data)

In [3]:
def merge_spaces(input_string):
    output_string = re.sub(r'\s+', ' ', input_string)
    return output_string

In [19]:
len(plos_datas)

1000

In [5]:
len(elife_datas)

241

In [56]:
sorted_datas = sorted(plos_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).strip().split(' ')))

In [84]:
sorted_datas = sorted(elife_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).strip().split(' ')))

In [25]:
plos_datas[395]

{'doi': '10.1371/journal.pntd.0004934',
 'title': 'Identifying Leprosy and Those at Risk of Developing Leprosy by Detection of Antibodies against LID-1 and LID-NDO',
 'abstract': 'Leprosy is caused by Mycobacterium leprae infection and remains a major public health problem in many areas of the world. Challenges to its timely diagnosis result in delay in treatment, which is usually associated with severe disability. Although phenolic glycolipid (PGL)-I has been reported as auxiliary diagnostic tool, currently there is no serological assay routinely used in leprosy diagnosis. The aim of this study was to evaluate the effectiveness of two related reagents, LID-1 and LID-NDO, for the detection of M. leprae infection. Sera from 98 leprosy patients, 365 household contacts (HHC) and 98 endemic controls from Rio Grande do Norte, Brazil, were evaluated. A subgroup of the HHC living in a hyperendemic area was followed for 7–10 years. Antigen-specific antibody responses were highest in multibacil

In [3]:
plos_exp_query_recs = {}
plos_pls_query_recs = {}

from collections import defaultdict

In [6]:
elife_qg_prompt = {
    "prompt_input_pls": "Please propose 10 layman questions that are diverse and cover all parts of the following paper from eLife dataset.\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please propose 10 expert questions that are diverse and cover all parts of the following paper from eLife dataset.\nBelow is a paper.\n{context}\nNow the paper ends."
}

plos_qg_prompt = {
    "prompt_input_pls": "Please propose 10 layman questions that are diverse and cover all parts of the following paper from PLOS dataset.\nBelow is a paper.\n{context}\nNow the paper ends.",
    "prompt_input_exp": "Please propose 10 expert questions that are diverse and cover all parts of the following paper from PLOS dataset.\nBelow is a paper.\n{context}\nNow the paper ends."
}

In [8]:
def apply_prompt_elife_qg_pls_template(context,qa_pairs):
    return {
        "instruction": elife_qg_prompt['prompt_input_pls'].format(context=context.replace('\n',' ').strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }
def apply_prompt_elife_qg_exp_template(context,qa_pairs):
    return {
        "instruction": elife_qg_prompt['prompt_input_exp'].format(context=context.replace('\n',' ').strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }

def apply_prompt_plos_qg_pls_template(context,qa_pairs):
    return {
        "instruction": plos_qg_prompt['prompt_input_pls'].format(context=merge_spaces(context.replace('\n',' ')).strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }
def apply_prompt_plos_qg_exp_template(context,qa_pairs):
    return {
        "instruction": plos_qg_prompt['prompt_input_exp'].format(context=merge_spaces(context.replace('\n',' ')).strip()),
        "input": "",
        "output": qa_pairs.strip(),
        "history": []
    }

In [54]:
sorted_datas[95]

{'doi': '10.1371/journal.ppat.1000156',
 'title': 'BSE Case Associated with Prion Protein Gene Mutation',
 'abstract': 'Bovine spongiform encephalopathy (BSE) is a transmissible spongiform encephalopathy (TSE) of cattle and was first detected in 1986 in the United Kingdom. It is the most likely cause of variant Creutzfeldt-Jakob disease (CJD) in humans. The origin of BSE remains an enigma. Here we report an H-type BSE case associated with the novel mutation E211K within the prion protein gene (Prnp). Sequence analysis revealed that the animal with H-type BSE was heterozygous at Prnp nucleotides 631 through 633. An identical pathogenic mutation at the homologous codon position (E200K) in the human Prnp has been described as the most common cause of genetic CJD. This finding represents the first report of a confirmed case of BSE with a potential pathogenic mutation within the bovine Prnp gene. A recent epidemiological study revealed that the K211 allele was not detected in 6062 cattle fr

In [4]:
import re
# sum = 0
# with open("plos_pls_query.txt") as f:
with open("pls_test_100.txt") as f:
    
    for nid, cline in enumerate(f.readlines()):
        
        # cur_dict = {}
        
        # assert sorted_datas[nid]['doi'] == cline.split('\t')[0], cline.split('\t')[0]
        
        if cline.split('\t')[-1][:2] == "1.":
            vline = "1:" + cline.split('\t')[-1][2:]
        else:
            vline = cline.split('\t')[-1]
            
        for i in range(2,11):
            vline = vline.replace(f".{i}. ", f".{i}: ").replace(f"{i}: ", f".{i}: ")
        # print(vline)
        
        line = re.sub(r'\b\d+: ', '\nQuestion: ', vline).replace('Answer:'," Answer:").strip().replace(' Answer:',"\nAnswer:")
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        # print(cline)
        # print(len(line.split('\n')))
        
        # assert len(line.split('\n')) % 2 == 0, cline
        if  len(line.split('\n')) != 20:
            # print(cline,line.split('\n'))
            print(nid)
            continue
        
        # sum +=1
        my_list = line.split('\n')
        # paired_list = [(my_list[i].strip(), re.sub(r'\.{2,}$', '.', my_list[i+1].strip())) for i in range(0, len(my_list), 2)]
        querys_list = [my_list[i].strip() for i in range(0, len(my_list), 2)]
        
        # cur_dict[merge_spaces(sorted_datas[nid]['article'].replace('\n',' ')).strip()] = paired_list
        # cur_dict[cline.split('\t')[0]] = paired_list
        
        plos_pls_query_recs[cline.split('\t')[0]] = querys_list
        
        # print(paired_list)

274
362
812


In [10]:
# sum = 0
# with open("plos_pls_query.txt") as f:
with open("exp_test_100.txt") as f:
    
    for nid, cline in enumerate(f.readlines()):
        
        # cur_dict = {}
        
        # assert sorted_datas[nid]['doi'] == cline.split('\t')[0], cline.split('\t')[0]
        
        if cline.split('\t')[-1][:2] == "1.":
            vline = "1:" + cline.split('\t')[-1][2:]
        else:
            vline = cline.split('\t')[-1]
            
        for i in range(2,11):
            vline = vline.replace(f".{i}. ", f".{i}: ").replace(f"{i}: ", f".{i}: ")
        # print(vline)
        
        line = re.sub(r'\b\d+: ', '\nQuestion: ', vline).replace('Answer:'," Answer:").strip().replace(' Answer:',"\nAnswer:")
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        # print(cline)
        # print(len(line.split('\n')))
        
        # assert len(line.split('\n')) % 2 == 0, cline
        if  len(line.split('\n')) != 20:
            # print(cline,line.split('\n'))
            continue
        
        # sum +=1
        my_list = line.split('\n')
        # paired_list = [(my_list[i].strip(), re.sub(r'\.{2,}$', '.', my_list[i+1].strip())) for i in range(0, len(my_list), 2)]
        querys_list = [my_list[i].strip() for i in range(0, len(my_list), 2)]
        
        # cur_dict[merge_spaces(sorted_datas[nid]['article'].replace('\n',' ')).strip()] = paired_list
        # cur_dict[cline.split('\t')[0]] = paired_list
        
        plos_exp_query_recs[cline.split('\t')[0]] = querys_list
        
        # print(paired_list)

In [11]:
elife_exp_query_recs = {}
elife_pls_query_recs = {}

# sum = 0
# with open("elife_pls_query.txt") as f:
with open("elife_pls_test_100.txt") as f:
    
    for nid, cline in enumerate(f.readlines()):
        
        # cur_dict = {}
        
        # assert sorted_datas[nid]['doi'] == cline.split('\t')[0], cline.split('\t')[0]
        
        if cline.split('\t')[-1][:2] == "1.":
            vline = "1:" + cline.split('\t')[-1][2:]
        else:
            vline = cline.split('\t')[-1]
            
        for i in range(2,11):
            vline = vline.replace(f".{i}. ", f".{i}: ").replace(f"{i}: ", f".{i}: ")
        # print(vline)
        
        line = re.sub(r'\b\d+: ', '\nQuestion: ', vline).replace('Answer:'," Answer:").strip().replace(' Answer:',"\nAnswer:")
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        # print(cline)
        # print(len(line.split('\n')))
        
        # assert len(line.split('\n')) % 2 == 0, cline
        if  len(line.split('\n')) != 20:
            # print(cline,line.split('\n'))
            continue
        
        # sum +=1
        my_list = line.split('\n')
        # paired_list = [(my_list[i].strip(), re.sub(r'\.{2,}$', '.', my_list[i+1].strip())) for i in range(0, len(my_list), 2)]
        querys_list = [my_list[i].strip() for i in range(0, len(my_list), 2)]
        
        # cur_dict[merge_spaces(sorted_datas[nid]['article'].replace('\n',' ')).strip()] = paired_list
        # cur_dict[cline.split('\t')[0]] = paired_list
        
        elife_pls_query_recs[cline.split('\t')[0]] = querys_list
        
        # print(paired_list)

In [12]:
# sum = 0
# with open("elife_exp_query.txt") as f:
with open("elife_exp_test_100.txt") as f:
    
    for nid, cline in enumerate(f.readlines()):
        
        # cur_dict = {}
        
        # assert sorted_datas[nid]['doi'] == cline.split('\t')[0]
        
        if cline.split('\t')[-1][:2] == "1.":
            vline = "1:" + cline.split('\t')[-1][2:]
        else:
            vline = cline.split('\t')[-1]
            
        for i in range(2,11):
            vline = vline.replace(f".{i}. ", f".{i}: ").replace(f"{i}: ", f".{i}: ")
        # print(vline)
        
        line = re.sub(r'\b\d+: ', '\nQuestion: ', vline).replace('Answer:'," Answer:").strip().replace(' Answer:',"\nAnswer:")
        # with open('./filter_doc.txt', 'a+', encoding='utf-8') as f:
        # print(cline)
        # print(len(line.split('\n')))
        
        # assert len(line.split('\n')) % 2 == 0, cline
        if  len(line.split('\n')) != 20:
            # print(cline,line.split('\n'))
            continue
        
        # sum +=1
        my_list = line.split('\n')
        # paired_list = [(my_list[i].strip(), re.sub(r'\.{2,}$', '.', my_list[i+1].strip())) for i in range(0, len(my_list), 2)]
        querys_list = [my_list[i].strip() for i in range(0, len(my_list), 2)]
        
        # cur_dict[merge_spaces(sorted_datas[nid]['article'].replace('\n',' ')).strip()] = paired_list
        # cur_dict[cline.split('\t')[0]] = paired_list
        
        elife_exp_query_recs[cline.split('\t')[0]] = querys_list
        
        # print(paired_list)

In [13]:
len(elife_pls_query_recs.keys()),len(elife_exp_query_recs.keys())

(241, 238)

In [14]:
len(plos_pls_query_recs.keys()),len(plos_exp_query_recs.keys())

(997, 996)

In [15]:
plos_sets = set(plos_exp_query_recs.keys()).intersection(set(plos_pls_query_recs.keys()))

In [16]:
elife_sets = set(elife_exp_query_recs.keys()).intersection(set(elife_pls_query_recs.keys()))

In [17]:
len(plos_sets)

994

In [18]:
len(elife_sets)

238

In [21]:
# Query Generation: 
# plos_exp_query_pairs，plos_pls_query_pairs，elife_pls_query_pairs, elife_exp_query_pairs （id，qa-pairs）
from tqdm import tqdm

# pdata_lists = []
pdata_qu_lists = []
# elife_sets, plos_sets 
# 
# sorted_datas = sorted(plos_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).split(' ')))[:3000]
# indexed_datas = {sorted_data["doi"]:sorted_data for sorted_data in sorted_datas}
indexed_datas = {plos_data["doi"]:plos_data for plos_data in plos_datas}
# for data in clip_plos_datas:
#     data_lists.append(apply_prompt_plos_query_exp_template(data))
#     data_lists.append(apply_prompt_plos_query_pls_template(data))
# for data in clip_elife_datas:
#     data_lists.append(apply_prompt_elife_query_exp_template(data))
#     data_lists.append(apply_prompt_elife_query_pls_template(data))
# for plos_pair in plos_exp_query_pairs:
#     if 
# list(elife_sets)[0],list(plos_sets)[0]

for nid, plos_id in enumerate(tqdm(list(plos_sets))):
    
    #TODO: 
    exp_qus = plos_exp_query_recs[plos_id]
    pls_qus = plos_pls_query_recs[plos_id]
    
    # context = merge_spaces(sorted_datas[plos_id]['article'].replace('\n',' ')).strip()
    context = merge_spaces(indexed_datas[plos_id]['article'].replace('\n',' ')).strip()
    
    
    # exp_seq = " ".join(exp_qus)
    # pls_seq = " ".join(pls_qus)
    exp_seq = []
    pls_seq = []
    for q_i in range(len(exp_qus)):
        exp_seq.append(str(q_i+1) + ": " + exp_qus[q_i].replace("Question:","").strip())
    for q_i in range(len(pls_qus)):
        pls_seq.append(str(q_i+1) + ": " + pls_qus[q_i].replace("Question:","").strip())
    
    pdata_qu_lists.append(apply_prompt_plos_qg_exp_template(context," ".join(exp_seq)))
    pdata_qu_lists.append(apply_prompt_plos_qg_pls_template(context," ".join(pls_seq)))

100%|██████████| 994/994 [00:06<00:00, 143.97it/s]


In [22]:
# Query Generation: 
# plos_exp_query_pairs，plos_pls_query_pairs，elife_pls_query_pairs, elife_exp_query_pairs （id，qa-pairs）
from tqdm import tqdm

# pdata_lists = []
edata_qu_lists = []
# elife_sets, plos_sets 
# 
# sorted_datas = sorted(plos_datas, key=lambda x: len(merge_spaces(x['article'].replace('\n',' ')).split(' ')))[:3000]
# indexed_datas = {sorted_data["doi"]:sorted_data for sorted_data in sorted_datas}
indexed_datas = {elife_data["doi"]:elife_data for elife_data in elife_datas}
# for data in clip_plos_datas:
#     data_lists.append(apply_prompt_plos_query_exp_template(data))
#     data_lists.append(apply_prompt_plos_query_pls_template(data))
# for data in clip_elife_datas:
#     data_lists.append(apply_prompt_elife_query_exp_template(data))
#     data_lists.append(apply_prompt_elife_query_pls_template(data))
# for plos_pair in plos_exp_query_pairs:
#     if 
# list(elife_sets)[0],list(plos_sets)[0]

for nid, elife_id in enumerate(tqdm(list(elife_sets))):
    
    exp_qus = elife_exp_query_recs[elife_id]
    pls_qus = elife_pls_query_recs[elife_id]
    
    # context = merge_spaces(sorted_datas[plos_id]['article'].replace('\n',' ')).strip()
    context = merge_spaces(indexed_datas[elife_id]['article'].replace('\n',' ')).strip()
    
    # exp_seq = " ".join(exp_qus)
    # pls_seq = " ".join(pls_qus)
    exp_seq = []
    pls_seq = []
    for q_i in range(len(exp_qus)):
        exp_seq.append(str(q_i+1) + ": " + exp_qus[q_i].replace("Question:","").strip())
    for q_i in range(len(pls_qus)):
        pls_seq.append(str(q_i+1) + ": " + pls_qus[q_i].replace("Question:","").strip())
    
    edata_qu_lists.append(apply_prompt_elife_qg_exp_template(context," ".join(exp_seq)))
    edata_qu_lists.append(apply_prompt_elife_qg_pls_template(context," ".join(pls_seq)))

100%|██████████| 238/238 [00:00<00:00, 257.55it/s]


In [26]:

length = min(len(pdata_qu_lists), len(edata_qu_lists))

mixed_list = []

for i in range(0, length, 4):
    mixed_list.extend(pdata_qu_lists[i:i+4])
    mixed_list.extend(edata_qu_lists[i:i+4])
    # mixed_list.extend(edata_lists[i:i+4])
    # mixed_list.extend(edata_qa_lists[i:i+4])

mixed_list.extend(pdata_qu_lists[length:])
mixed_list.extend(edata_qu_lists[length:])

In [27]:
len(mixed_list)

2464

In [ ]:
file_path = 'qg.json'

with open(file_path, 'w') as file:
    json.dump(mixed_list, file)


JSON 文件写入成功！


In [98]:
len(edata_qu_lists),len(pdata_qu_lists)

(5958, 5946)

In [88]:
list(elife_sets)[0]

'elife-28197-v1'

In [12]:
with open("plos_pls_query_com.txt") as f:
    b = f.readlines()
len(b)

3000

In [10]:
with open("plos_exp_query.txt") as f:
    a = f.readlines()

with open("plos_exp_query_ori.txt") as f:
    b = f.readlines()
    

content_b_first = b[:2138]

content_a_from = a[2138:]


combined_content = content_b_first + content_a_from


with open("plos_exp_query_com.txt", "w") as output_file:
    output_file.writelines(combined_content)


文件合并完成并写入combined_file.txt
